In [113]:
import pandas as pd

# TODO:
# - keep patientnrs (add for blood bank)

In [135]:
other_imid = pd.read_spss("../data/interim/OtherIMID.sav") 
all = pd.read_spss("../data/interim/Alles 1e sample.sav") # SLE patients (also has non- and other-IMID data)
non_imid = pd.read_spss("../data/interim/Non-Imid control set.sav")
tmo = pd.read_csv("../data/interim/TMO.csv") # blood bank controls (also has data from SLE patients)

# load translation table of columns in TMO.csv vs. the .sav files
df_cols = pd.read_csv("../references/chip_colnames.csv", sep=";")  

In [132]:
sle = all[all.SLE == 1] # keep only SLE patients
blood_bank = tmo[tmo.Class == "nonSLE"] # keep only blood baknk controls

In [150]:
df_cols = df_cols.dropna() # keep only names of columns that exist in both data sets
new_colnames = df_cols.TB.tolist() # keep names as in .sav files
new_colnames.append('Class') # extra column in blood bank data (we'll need it in the others as well)
blood_bank.columns = new_colnames # rename columns as in other datasets

In [154]:
other_imid['Class'] = "IMID" 
other_imid = other_imid[new_colnames] # keep only columns that also occur in blood_bank

non_imid['Class'] = "nonIMID"
non_imid = non_imid[new_colnames]

sle['Class'] = "SLE"
sle = sle[new_colnames]

C:\ProgramData\Miniconda3\envs\SLE\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Miniconda3\envs\SLE\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [155]:
# join all data frames together by binding rows
df_all = pd.concat([sle, other_imid, non_imid, blood_bank])